In [26]:
from pandasql import sqldf
import pandas as pd

pysqldf = lambda q: sqldf(q,globals())


In [27]:
sql = """
CREATE TABLE clients (
    client_id       INT NOT NULL,
    firstname       VARCHAR(200),
    lastname        VARCHAR(200),
    home_address_id INT
);
"""

In [28]:
clients = pd.DataFrame([],columns=['client_id','firstname','lastname','home_address_id'],)
clients['client_id'] = pd.to_numeric(clients.client_id)
clients['home_address_id'] = pd.to_numeric(clients.home_address_id)

In [29]:
clients.info()

<class 'pandas.core.frame.DataFrame'>
Index: 0 entries
Data columns (total 4 columns):
client_id          0 non-null int64
firstname          0 non-null object
lastname           0 non-null object
home_address_id    0 non-null int64
dtypes: int64(2), object(2)
memory usage: 0.0+ bytes


In [30]:
clients

,client_id,firstname,lastname,home_address_id


In [31]:
sql = """
INSERT INTO clients (client_id, firstname, lastname, home_address_id)
VALUES
    (102,'Mikel','Rouse',1002),
    (103,'Laura','Gibson',1003),
    (104,None,'Hurst',1003);
"""

In [32]:
data = [[102,'Mikel','Rouse',1002],[103,'Laura','Gibson',1003],[104,None,'Hurst',1003]]
for idx,row in enumerate(data):
    clients.loc[idx] = row

In [33]:
print(clients)

   client_id firstname lastname  home_address_id
0        102     Mikel    Rouse             1002
1        103     Laura   Gibson             1003
2        104      None    Hurst             1003


In [34]:
sql = """
SELECT 
    client_id
    ,lastname
FROM
    clients
"""
print(pysqldf(sql))

   client_id lastname
0        102    Rouse
1        103   Gibson
2        104    Hurst


In [35]:
print(clients.loc[:,['client_id','lastname']])

   client_id lastname
0        102    Rouse
1        103   Gibson
2        104    Hurst


In [36]:
sql = """
SELECT 
    client_id AS cid
    ,lastname
FROM
    clients
"""
print(pysqldf(sql))

   cid lastname
0  102    Rouse
1  103   Gibson
2  104    Hurst


In [37]:
print(clients.loc[:,['client_id','lastname']].rename({'client_id':'cid'},axis=1))

   cid lastname
0  102    Rouse
1  103   Gibson
2  104    Hurst


In [38]:
sql = """
SELECT 
    *
FROM
    clients
"""
print(pysqldf(sql))

   client_id firstname lastname  home_address_id
0        102     Mikel    Rouse             1002
1        103     Laura   Gibson             1003
2        104      None    Hurst             1003


In [25]:
print(clients)

   client_id first_name last_name  home_address_id
0        102      Mikel     Rouse             1002
1        103      Laura    Gibson             1003
2        104       None     Hurst             1003


In [49]:
sql = """
SELECT 
    *
FROM
    clients
WHERE home_address_id = 1003
"""
print(pysqldf(sql))

   client_id firstname lastname  home_address_id
0        103     Laura   Gibson             1003
1        104      None    Hurst             1003


In [51]:
print(clients[clients.home_address_id == 1003])

   client_id firstname lastname  home_address_id
1        103     Laura   Gibson             1003
2        104      None    Hurst             1003


In [40]:
sql = """
SELECT 
    *
FROM
    clients
WHERE lastname LIKE 'Gi%'
"""
print(pysqldf(sql))

   client_id firstname lastname  home_address_id
0        103     Laura   Gibson             1003


In [42]:
print(clients[clients.lastname.str.startswith('Gi')])

   client_id firstname lastname  home_address_id
1        103     Laura   Gibson             1003


In [47]:
sql = """
SELECT 
    firstname
    ,lastname
FROM
    clients
WHERE lastname LIKE 'Gi%'
"""
print(pysqldf(sql))

  firstname lastname
0     Laura   Gibson


In [48]:
print(clients.loc[clients.lastname.str.startswith('Gi'),['firstname','lastname']])

  firstname lastname
1     Laura   Gibson


In [54]:
sql = """
SELECT 
    *
FROM
    clients
WHERE home_address_id = 1003 AND lastname LIKE 'Gi%'
"""
print(pysqldf(sql))

   client_id firstname lastname  home_address_id
0        103     Laura   Gibson             1003


In [56]:
print(clients.loc[(clients.home_address_id == 1003) & 
                  (clients.lastname.str.startswith('Gi'))])

   client_id firstname lastname  home_address_id
1        103     Laura   Gibson             1003


In [81]:
sql = """
SELECT
    DISTINCT
    home_address_id
FROM clients
"""
print(pysqldf(sql))

   home_address_id
0             1002
1             1003


In [83]:
print(clients.home_address_id.drop_duplicates().to_frame())

   home_address_id
0             1002
1             1003


In [85]:
sql = """
SELECT
    firstname
    ,lastname
FROM clients
LIMIT 1
"""
print(pysqldf(sql))

  firstname lastname
0     Mikel    Rouse


In [94]:
print(clients.loc[:,['firstname','lastname']].iloc[:1,:])

  firstname lastname
0     Mikel    Rouse


In [72]:
sql = """
SELECT 
    -- the concatenation operator is ||
    client_id
    ,firstname || ' ' || lastname AS fullname
FROM
    clients
WHERE firstname IS NOT NULL
"""

print(pysqldf(sql))

   client_id      fullname
0        102   Mikel Rouse
1        103  Laura Gibson


In [79]:
fullname = lambda x:x['firstname'] + ' ' + x['lastname']
tmp = clients.dropna(subset=['firstname'])[['client_id']]
tmp['fullname'] = clients.dropna(subset=['firstname']).apply(fullname,axis=1)
print(tmp)

   client_id      fullname
0        102   Mikel Rouse
1        103  Laura Gibson


In [109]:
sql = """
SELECT
    COUNT(*) AS record_count
FROM clients
WHERE firstname IS NOT NULL
"""
print(pysqldf(sql))

   record_count
0             2


In [114]:
print(pd.DataFrame([clients.dropna(subset=['firstname']).shape[0]],columns=['record_count']))

   record_count
0             2


In [100]:
sql = """
SELECT
    *
FROM clients
ORDER BY lastname DESC
"""
print(pysqldf(sql))

   client_id firstname lastname  home_address_id
0        102     Mikel    Rouse             1002
1        104      None    Hurst             1003
2        103     Laura   Gibson             1003


In [102]:
print(clients.sort_values(by='lastname',ascending=False))

   client_id firstname lastname  home_address_id
0        102     Mikel    Rouse             1002
2        104      None    Hurst             1003
1        103     Laura   Gibson             1003


In [134]:
sql = """
SELECT 
    home_address_id
    ,COUNT(*) as nclients
FROM clients
GROUP BY home_address_id
"""
print(pysqldf(sql))

   home_address_id  nclients
0             1002         1
1             1003         2


In [139]:
print(clients.groupby('home_address_id').client_id.count().\
      reset_index().rename({'client_id':'nclients'},axis=1))

   home_address_id  nclients
0             1002         1
1             1003         2


In [144]:
sql = """
SELECT
    subquery.home_address_id
    ,subquery.nclients
FROM (
    SELECT 
        home_address_id
        ,COUNT(*) as nclients
    FROM clients
    GROUP BY home_address_id
) subquery
WHERE nclients > 1

"""
print(pysqldf(sql))

   home_address_id  nclients
0             1003         2


In [147]:
tmp = clients.groupby('home_address_id').client_id.count().\
          reset_index().rename({'client_id':'nclients'},axis=1)
print(tmp[tmp.nclients > 1])

   home_address_id  nclients
1             1003         2


In [148]:
addresses = pd.DataFrame([[1002,'1 First Ave.'],[1003,'2 Second Ave.']],columns=['address_id','address'])

In [151]:
sql = """
SELECT
    c.firstname
    ,a.address
FROM clients AS c
JOIN addresses AS a ON c.home_address_id = a.address_id
WHERE c.firstname IS NOT NULL
"""
print(pysqldf(sql))

  firstname        address
0     Mikel   1 First Ave.
1     Laura  2 Second Ave.


In [162]:
print(pd.merge(clients.dropna(),
               addresses, 
               left_on='home_address_id',
               right_on='address_id').loc[:,['firstname','address']])

  firstname        address
0     Mikel   1 First Ave.
1     Laura  2 Second Ave.


In [165]:
clients.loc[3] = [105,'Scott','Payseur',1004]

In [166]:
sql = """
SELECT
    c.firstname
    ,a.address
FROM clients AS c
LEFT JOIN addresses AS a ON c.home_address_id = a.address_id
WHERE c.firstname IS NOT NULL
"""
print(pysqldf(sql))

  firstname        address
0     Mikel   1 First Ave.
1     Laura  2 Second Ave.
2     Scott           None


In [167]:
print(pd.merge(clients.dropna(),
               addresses, 
               left_on='home_address_id',
               right_on='address_id',
               how='left'
              ).loc[:,['firstname','address']])

  firstname        address
0     Mikel   1 First Ave.
1     Laura  2 Second Ave.
2     Scott            NaN


In [168]:
addresses.loc[2] = [1005,'3 Third Ave.']

In [169]:
sql = """
SELECT
    c.firstname
    ,a.address
FROM clients AS c
RIGHT JOIN addresses AS a ON c.home_address_id = a.address_id
WHERE c.firstname IS NOT NULL
"""
print(pysqldf(sql))

PandaSQLException: (sqlite3.OperationalError) RIGHT and FULL OUTER JOINs are not currently supported
[SQL: 
SELECT
    c.firstname
    ,a.address
FROM clients AS c
RIGHT JOIN addresses AS a ON c.home_address_id = a.address_id
WHERE c.firstname IS NOT NULL
]
(Background on this error at: http://sqlalche.me/e/e3q8)

In [170]:
print(pd.merge(clients.dropna(),
               addresses, 
               left_on='home_address_id',
               right_on='address_id',
               how='right'
              ).loc[:,['firstname','address']])

  firstname        address
0     Mikel   1 First Ave.
1     Laura  2 Second Ave.
2       NaN   3 Third Ave.


In [171]:
sql = """
SELECT
    c.firstname
    ,a.address
FROM clients AS c
OUTER JOIN addresses AS a ON c.home_address_id = a.address_id
WHERE c.firstname IS NOT NULL
"""
print(pysqldf(sql))

PandaSQLException: (sqlite3.OperationalError) RIGHT and FULL OUTER JOINs are not currently supported
[SQL: 
SELECT
    c.firstname
    ,a.address
FROM clients AS c
OUTER JOIN addresses AS a ON c.home_address_id = a.address_id
WHERE c.firstname IS NOT NULL
]
(Background on this error at: http://sqlalche.me/e/e3q8)

In [174]:
print(
pd.merge(clients.dropna(), addresses, 
         left_on='home_address_id', right_on='address_id',
         how='outer'
        ).loc[:,['firstname','address']]
)

  firstname        address
0     Mikel   1 First Ave.
1     Laura  2 Second Ave.
2     Scott            NaN
3       NaN   3 Third Ave.


In [177]:
clients['bus_address_id'] = [1002,1004,1005,1005]

In [185]:
sql = """
SELECT 
    c.firstname || ' ' || c.lastname AS fullname
    ,ha.address AS home_address
    ,ba.address AS bus_address
FROM
    clients AS c
LEFT JOIN addresses AS ha ON ha.address_id = c.home_address_id
LEFT JOIN addresses AS ba ON ba.address_id = c.bus_address_id
WHERE c.firstname IS NOT NULL
ORDER BY fullname
"""
print(pysqldf(sql))

        fullname   home_address   bus_address
0   Laura Gibson  2 Second Ave.          None
1    Mikel Rouse   1 First Ave.  1 First Ave.
2  Scott Payseur           None  3 Third Ave.


In [287]:
df = pd.read_csv('../data/yellow_tripdata_2017-01_subset10000rows.csv',
                 parse_dates=['tpep_pickup_datetime','tpep_dropoff_datetime'])

In [288]:
df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
0,2,2017-01-10 18:37:59,2017-01-10 18:49:24,1,0.71,1,N,162,230,2,8.0,1.0,0.5,0.00,0.0,0.3,9.80
1,2,2017-01-05 15:14:52,2017-01-05 15:23:28,1,1.16,1,N,142,239,1,7.5,0.0,0.5,1.66,0.0,0.3,9.96
2,2,2017-01-11 14:47:52,2017-01-11 14:59:15,1,1.30,1,N,237,75,1,8.5,0.0,0.5,1.00,0.0,0.3,10.30
3,1,2017-01-04 07:31:49,2017-01-04 07:39:21,1,0.80,1,N,262,140,2,6.5,0.0,0.5,0.00,0.0,0.3,7.30
4,1,2017-01-12 17:43:50,2017-01-12 17:51:25,1,1.40,1,N,90,114,2,7.0,1.0,0.5,0.00,0.0,0.3,8.80


In [289]:
import sqlalchemy

engine = sqlalchemy.create_engine('sqlite:///yellowtaxi_db')

In [290]:
df.to_sql('trips',engine,if_exists='replace',index=True,index_label='trip_id')

In [291]:
sql = """
SELECT 
    tpep_pickup_datetime AS pickup_time
    ,fare_amount + tip_amount AS total_amount 
FROM trips 
WHERE trip_distance > 25 
ORDER BY fare_amount DESC 
LIMIT 5
"""

In [298]:
print(
pd.read_sql(sql,engine)
)

                  pickup_time  total_amount
0  2017-01-06 08:43:10.000000        219.00
1  2017-01-23 15:52:48.000000        165.00
2  2017-01-03 14:55:31.000000        159.00
3  2017-01-19 17:41:45.000000        150.36
4  2017-01-02 06:28:47.000000         88.00


In [299]:
df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
0,2,2017-01-10 18:37:59,2017-01-10 18:49:24,1,0.71,1,N,162,230,2,8.0,1.0,0.5,0.00,0.0,0.3,9.80
1,2,2017-01-05 15:14:52,2017-01-05 15:23:28,1,1.16,1,N,142,239,1,7.5,0.0,0.5,1.66,0.0,0.3,9.96
2,2,2017-01-11 14:47:52,2017-01-11 14:59:15,1,1.30,1,N,237,75,1,8.5,0.0,0.5,1.00,0.0,0.3,10.30
3,1,2017-01-04 07:31:49,2017-01-04 07:39:21,1,0.80,1,N,262,140,2,6.5,0.0,0.5,0.00,0.0,0.3,7.30
4,1,2017-01-12 17:43:50,2017-01-12 17:51:25,1,1.40,1,N,90,114,2,7.0,1.0,0.5,0.00,0.0,0.3,8.80


In [300]:
pd.read_sql('SELECT * FROM trips LIMIT 1',engine,index_col='trip_id')

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
trip_id,,,,,,,,,,,,,,,,,
0,2,2017-01-10 18:37:59.000000,2017-01-10 18:49:24.000000,1,0.71,1,N,162,230,2,8.0,1.0,0.5,0.0,0.0,0.3,9.8


In [301]:
import pymongo

ModuleNotFoundError: No module named 'pymongo'